In [82]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
import matplotlib.pyplot as plt

print('importing dataset..')
frame=pd.read_csv('train.csv',delimiter=',')
print('Importing completed !!')
cols=['polarity','ID','date','query','user','tweet']
frame.columns=cols
del frame['ID']
del frame['date']
del frame['user']
polarity=frame.polarity
tweet=frame.tweet
X_train, X_test, y_train, y_test = train_test_split(tweet, polarity, test_size=0.5)
# try test_size=0.5 ,0.25 and 0.33
def tweet_to_words(raw_review):
        ex=raw_review
        # now we import BeautifulSoup to remove the tags and markups from the text data 
        soup=BeautifulSoup(ex)
        clean_text=soup.get_text()
        # Dealing with Punctuations
        # Use regular expressions to do a find-and-replace
        letters_only=re.sub("[^a-zA-Z]", " ", clean_text)
        # tokenizing
        lower_case=letters_only.lower()
        words=lower_case.split()
        # remove stopwords
        stops=set(stopwords.words('english'))
        words= [w for w in words if not w in stops]
        #Stemming
        lancaster_stemmer=LancasterStemmer()
        stem_word=[]
        for word in words:
            stem_word.append(lancaster_stemmer.stem(word))
            new_set=" ".join( stem_word )
        return new_set
    
num_reviews = X_train.size
print "Cleaning and parsing the training set tweets...\n"
clean_train_tweets = []
X_train=X_train.tolist()
for i in range(num_reviews ):
    # If the index is evenly divisible by 100, print a message
    if( (i+1)%100 == 0 ):
        print "Review %d of %d\n" % ( i+1, num_reviews )                                                                    
    clean_train_tweets.append( tweet_to_words( X_train[i] ))
 

num_reviews2=X_test.size
print "Cleaning and parsing the test set tweets...\n"
clean_test_tweets = []
X_test=X_test.tolist()
for i in range(num_reviews2 ):
    # If the index is evenly divisible by 100, print a message
    if( (i+1)%100 == 0 ):
        print "Review %d of %d\n" % ( i+1, num_reviews2 )                                                                    
    clean_test_tweets.append( tweet_to_words( X_test[i] ))
# this function calculates the probability of a person belonging to a particular personality on the basis of polarity of his tweets
def personality_predictor(polarity):
    polarity=list(polarity)
    a=len(polarity)
    pos_count=polarity.count(4)
    neutral_count=polarity.count(2)
    neg_count=polarity.count(0)
    personality=[]
    if ((pos_count/a)>0.5):
        personality.append('Enthusiastic and')
    elif ((neg_count/a)>0.5):
        personality.append('Apathetic and ')
    elif (neutral_count/a>0.5):
        personality.append('Pretty Calm and ')
    else:
        personality.append('Balanced being and')
    
    if ((pos_count+neutral_count)>0.5):
        personality.append('Trustable and Optimist')
    
    elif ((pos_count+neutral_count)<0.5):
            personality.append('Judgemental,Harsh')
    else:
            personality.append('Pessimistic')
            
    return personality
   
result=personality_predictor(y_test)
print (' Acoording to training set tweets the person is %s %s also. \n') % (result[0],result[1])
#feature Creation
from sklearn.feature_extraction.text import CountVectorizer
#CountVectorizer: Scikit learn's bag of tool
vectorizer=CountVectorizer(analyzer='word', tokenizer=None,preprocessor=None,stop_words=None,max_features=500)
train_data_features=vectorizer.fit_transform(clean_train_tweets)
train_data_features= train_data_features.toarray()
# Random Forest
print "Training the random forest...."
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(n_estimators=100)
forest=forest.fit(train_data_features,y_train)
# Use the random forest to make sentiment label predictions
test_data_features=vectorizer.fit_transform(clean_test_tweets)
test_data_features= test_data_features.toarray()
result2 = forest.predict(test_data_features)
result3=personality_predictor(result2)
print (' Acoording to test set tweets the person is %s and %s also. \n') % (result3[0],result3[1])

# The slices will be ordered and plotted counter-clockwise.
labels = 'Positive Tweets', 'Neutral Tweets', 'Negative Tweets'
y_train=list(y_train)
train_pos=y_train.count(4)
train_neu=y_train.count(2)
train_neg=y_train.count(0)
train_sizes = [train_pos,train_neu, train_neg]

y_test=list(y_test)
test_pos=y_test.count(4)
test_neu=y_test.count(2)
test_neg=y_test.count(0)
test_sizes = [test_pos,test_neu, test_neg]

result2=list(result2)
pred_pos=result2.count(4)
pred_neu=result2.count(2)
pred_neg=result2.count(0)
pred_sizes = [pred_pos,pred_neu,pred_neg]
colors = ['yellowgreen', 'gold', 'lightskyblue']
explode = (0.1, 0, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')
# Set aspect ratio to be equal so that pie is drawn as a circle.
fig = plt.figure()
ax = fig.gca()
import numpy as np
ax.pie(train_sizes, explode=explode, labels=labels, colors=colors,
       autopct='%1.1f%%', shadow=True, startangle=90,
       radius=0.25, center=(0, 0), frame=True)
ax.pie(test_sizes, explode=explode, labels=labels, colors=colors,
       autopct='%1.1f%%', shadow=True, startangle=90,
       radius=0.25, center=(1, 1), frame=True)
ax.pie(pred_sizes, explode=explode, labels=labels, colors=colors,
       autopct='%1.1f%%', shadow=True, startangle=90,
       radius=0.25, center=(0, 1), frame=True)

# Set aspect ratio to be equal so that pie is drawn as a circle.
ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels(["Predicted Tweets", ""])
ax.set_yticklabels(["", "Training set tweets"])
ax.set_xlim((-0.5, 1.5))
ax.set_ylim((-0.5, 1.5))
plt.show()

importing dataset..
Importing completed !!
Cleaning and parsing the training set tweets...

Review 100 of 248

Review 200 of 248

Cleaning and parsing the test set tweets...

Review 100 of 249

Review 200 of 249

 Acoording to training set tweets the person is Balanced being and Trustable and Optimist also. 

Training the random forest....
 Acoording to test set tweets the person is Balanced being and and Trustable and Optimist also. 

